Решение:

- Заместо парсинга будем решать задачу zero-shot классификации
- Для каждого класса подобрали самый лучший prompt
- Использовали CLIP модель (apple/DFN5B-CLIP-ViT-H-14-378), чтобы выбрать картинки близкие к каждому промтпу-тексту
- Класс "Остальное" назначали тем картинкам, у которых softmax threshold <= 0.2

Что ещё пробовали:

- Перебирали различные промпты
- Пробовали добавлять доп. классы, чтобы лучше отсеивать 16-ый класс
- Модели:
    - apple/DFN5B-CLIP-ViT-H-14-378
    - openai/clip-vit-large-patch14
    - openai/clip-vit-base-patch16
    - openai/clip-vit-large-patch14-336
    - openai/clip-vit-base-patch32
    - laion/CLIP-ViT-H-14-laion2B-s32B-b79K
    - google/siglip-so400m-patch14-384

Что ещё можно было попробовать:

- Кластеризация-классификация изображений с помощью ResNet с опорными изображениями
- Реально спарсить данные, но на многих сайтах они даны врассыпную и без категорий. Тогда всё равно пришлось бы использовать CLIP.
- Взять тренировочный датасет из [этого репозитория](https://github.com/omega-rg/Hotelify-A-classifier-for-hotel-images), там совпадают многие классы, но не ясно - улучшиться ли качество по сравнению с CLIP.

In [ ]:
!pip install gdown -q

In [ ]:
!gdown 1WhPxqM5zCQLV2YXzs_yagLM3MQk_t6SD
!unzip -o /content/private_test.zip

Downloading...
From (original): https://drive.google.com/uc?id=1WhPxqM5zCQLV2YXzs_yagLM3MQk_t6SD
From (redirected): https://drive.google.com/uc?id=1WhPxqM5zCQLV2YXzs_yagLM3MQk_t6SD&confirm=t&uuid=d20c5df0-5b71-4c8b-bf54-f3f101bd770e
To: /content/private_test.zip
100% 109M/109M [00:06<00:00, 17.6MB/s]
Archive:  /content/private_test.zip
  inflating: private_test/private_test_1.jpg  
  inflating: private_test/private_test_10.jpg  
  inflating: private_test/private_test_100.jpg  
  inflating: private_test/private_test_1000.jpg  
  inflating: private_test/private_test_1001.jpg  
  inflating: private_test/private_test_1002.jpg  
  inflating: private_test/private_test_1003.jpg  
  inflating: private_test/private_test_1004.jpg  
  inflating: private_test/private_test_1005.jpg  
  inflating: private_test/private_test_1006.jpg  
  inflating: private_test/private_test_1007.jpg  
  inflating: private_test/private_test_1008.jpg  
  inflating: private_test/private_test_1009.jpg  
  inflating: priva

In [ ]:
import os

import pandas as pd
from transformers import pipeline

folder_path = "/content/private_test"

class_labels = [
    "Hotel exterior, outdoor area, or building facade",
    "Hotel room, living space, or bedroom with furniture",
    "Swimming pool or hotel pool area",
    "Billiard table, pool table, or game room",
    "Bathroom with toilet, shower, sink, or bath amenities",
    "Hotel restaurant, dining room, or eating area",
    "Hotel lobby, reception area, or entrance hall",
    "Beachfront, shoreline, or sandy beach area",
    "Corridors, hallways, or staircases in the hotel",
    "Food dishes, meals on plates, or table settings",
    "Conference room, meeting room, or seminar space",
    "Gym, fitness center, or exercise equipment area",
    "Balcony view, outdoor balcony, or terrace",
    "Terrace, patio, or outdoor courtyard",
    "Spa, sauna, wellness center, or relaxation area",
]

image_classifier = pipeline(
    task="zero-shot-image-classification",
    model="apple/DFN5B-CLIP-ViT-H-14-378",
    device="cuda",
    batch_size=10,
)

image_files = [
    f"{folder_path}/{f}"
    for f in os.listdir(folder_path)
    if f.endswith(("jpg", "jpeg", "png"))
]
outputs = image_classifier(image_files, candidate_labels=class_labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

In [ ]:
threshold = 0.2

new_outputs = [x[0] for x in outputs]
scores = [x["score"] for x in new_outputs]
labels = [x["label"] for x in new_outputs]
df = pd.DataFrame({"images": image_files, "labels": labels, "scores": scores})

df["class_number"] = df["labels"].apply(lambda x: class_labels.index(x)) + 1
df["class_number"] = df.apply(
    lambda x: x["class_number"] if x["scores"] > threshold else 16,
    axis=1,
)
df["image_name"] = df["images"].apply(lambda x: x.split("/")[-1])
df = df.sort_values(
    by="image_name", key=lambda x: [int(y.split("_")[2].split(".")[0]) for y in x]
)

df[["image_name", "class_number"]].to_csv(
    f"apple-DFN5B-CLIP-ViT-H-14-378_{threshold}_top_prompts_with_ts.csv",
    index=False,
)
df

,images,labels,scores,class_number,image_name
864,/content/private_test/private_test_1.jpg,"Hotel lobby, reception area, or entrance hall",0.223197,7,private_test_1.jpg
922,/content/private_test/private_test_2.jpg,"Corridors, hallways, or staircases in the hotel",0.339878,9,private_test_2.jpg
779,/content/private_test/private_test_3.jpg,"Hotel room, living space, or bedroom with furn...",0.208196,2,private_test_3.jpg
818,/content/private_test/private_test_4.jpg,"Hotel room, living space, or bedroom with furn...",0.277408,2,private_test_4.jpg
439,/content/private_test/private_test_5.jpg,"Beachfront, shoreline, or sandy beach area",0.343005,8,private_test_5.jpg
...,...,...,...,...,...
970,/content/private_test/private_test_1131.jpg,"Hotel restaurant, dining room, or eating area",0.188961,16,private_test_1131.jpg
560,/content/private_test/private_test_1132.jpg,"Terrace, patio, or outdoor courtyard",0.175014,16,private_test_1132.jpg
937,/content/private_test/private_test_1133.jpg,"Spa, sauna, wellness center, or relaxation area",0.279637,15,private_test_1133.jpg
813,/content/private_test/private_test_1134.jpg,"Bathroom with toilet, shower, sink, or bath am...",0.549999,5,private_test_1134.jpg
